In [1]:
import geopandas as gpd

In [2]:
import pathlib
import random
from functools import reduce
from collections import defaultdict

In [3]:
import pandas as pd
import folium
import shapely
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import sklearn.cluster
import tensorflow as tf

In [4]:
import pydeck as pdk
import os

import cufflinks as cf 
cf.go_offline(connected=True)
cf.set_config_file(theme='polar')

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.rcParams["font.family"] = 'Nanum Gothic'

from shapely.geometry import Polygon, Point
from numpy import random

#최적화 solver
import time
from mip import Model, xsum, maximize, BINARY

In [5]:
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

def multipolygon_to_coordinates(x): 
    lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)]

In [6]:
# 격자별 현황
Nowon= gpd.read_file("C:/Users/user/Desktop/노원구/최종노원구.geojson") #데이터

Nowon

,gid,50대남,50대여,60대남,60대여,70대남,70대여,80대남,80대여,90대남,90대여,100대남,100대여,정보화,총교육,이용률,배움터,geometry
0,다사621606,35,35,20,11,0,6,0,7,0,0,0,0,86.728,88,1.000000,1.0,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ..."
1,다사621606,35,35,20,11,0,6,0,7,0,0,0,0,86.728,60,1.000000,1.0,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ..."
2,다사619593,60,69,32,41,21,35,9,33,0,0,0,0,73.970,83,0.285714,1.0,"MULTIPOLYGON (((127.06819 37.63237, 127.06819 ..."
3,다사621619,27,28,39,30,17,22,8,15,0,0,0,0,71.441,69,0.100000,1.0,"MULTIPOLYGON (((127.07033 37.65582, 127.07032 ..."
4,다사609604,0,0,0,0,0,0,0,0,0,0,0,0,0.000,54,1.000000,1.0,"MULTIPOLYGON (((127.05680 37.64225, 127.05680 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3739,다사601585,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0.000000,0.0,"MULTIPOLYGON (((127.04784 37.62509, 127.04783 ..."
3740,다사614591,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0.000000,0.0,"MULTIPOLYGON (((127.06254 37.63055, 127.06253 ..."
3741,다사625658,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0.000000,0.0,"MULTIPOLYGON (((127.07466 37.69098, 127.07465 ..."
3742,다사644652,0,0,0,0,0,0,0,0,0,0,0,0,0.000,0,0.000000,0.0,"MULTIPOLYGON (((127.09624 37.68565, 127.09623 ..."


In [7]:
Nowon['총인구수']=Nowon['50대남']+Nowon['50대여']+Nowon['60대남']+Nowon['60대여']+Nowon['70대남']+Nowon['70대여']+Nowon['80대남']+Nowon['80대여']+Nowon['90대남']+Nowon['90대여']+Nowon['100대남']+Nowon['100대여']

Nowon['정규화인구']=Nowon['총인구수']/Nowon['총인구수'].max()

Nowon.rename(columns={"정보화":"정보화점수",
                     '배움터':'디지털배움터수',
                     '총교육':'총교육수'},inplace=True)

Nowon=Nowon[['gid','총인구수','디지털배움터수','총교육수','이용률',
      '정보화점수','정규화인구','geometry']]

Nowon

,gid,총인구수,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,geometry
0,다사621606,114,1.0,88,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ..."
1,다사621606,114,1.0,60,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ..."
2,다사619593,300,1.0,83,0.285714,73.970,0.322581,"MULTIPOLYGON (((127.06819 37.63237, 127.06819 ..."
3,다사621619,186,1.0,69,0.100000,71.441,0.200000,"MULTIPOLYGON (((127.07033 37.65582, 127.07032 ..."
4,다사609604,0,1.0,54,1.000000,0.000,0.000000,"MULTIPOLYGON (((127.05680 37.64225, 127.05680 ..."
...,...,...,...,...,...,...,...,...
3739,다사601585,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.04784 37.62509, 127.04783 ..."
3740,다사614591,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.06254 37.63055, 127.06253 ..."
3741,다사625658,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.07466 37.69098, 127.07465 ..."
3742,다사644652,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.09624 37.68565, 127.09623 ..."


In [42]:
# geometry를 coordinate 형태로 적용
Nowon['coordinates']=Nowon['geometry'].apply(multipolygon_to_coordinates)

Nowon

,gid,총인구수,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,geometry,coordinates
0,다사621606,114,2.0,88,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ...","[[127.07039292621874, 37.64409879150235], [127..."
1,다사621606,114,2.0,60,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ...","[[127.07039292621874, 37.64409879150235], [127..."
2,다사619593,300,1.0,83,0.285714,73.970,0.322581,"MULTIPOLYGON (((127.06819 37.63237, 127.06819 ...","[[127.06819374780068, 37.63237335644675], [127..."
3,다사621619,186,1.0,69,0.100000,71.441,0.200000,"MULTIPOLYGON (((127.07033 37.65582, 127.07032 ...","[[127.07032542462314, 37.655815929528295], [12..."
4,다사609604,0,1.0,54,1.000000,0.000,0.000000,"MULTIPOLYGON (((127.05680 37.64225, 127.05680 ...","[[127.05680166898755, 37.642245840217655], [12..."
...,...,...,...,...,...,...,...,...,...
3739,다사601585,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.04784 37.62509, 127.04783 ...","[[127.04783770740246, 37.62508638407275], [127..."
3740,다사614591,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.06254 37.63055, 127.06253 ...","[[127.06253771656016, 37.630549841543136], [12..."
3741,다사625658,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.07466 37.69098, 127.07465 ...","[[127.07465953346531, 37.69098365406477], [127..."
3742,다사644652,0,0.0,0,0.000000,0.000,0.000000,"MULTIPOLYGON (((127.09624 37.68565, 127.09623 ...","[[127.09623893248742, 37.685651520884775], [12..."


In [8]:
# 100X100 grid에서 central point 찾기
Nowon_list = []
Nowon_list2 = []

for i in Nowon['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    Nowon_list.append(cent)
    Nowon_list2.append(Point(cent[0]))
Nowon['coord_cent']=0
Nowon['geo_cent']=0
Nowon['coord_cent']= pd.DataFrame(Nowon_list)
Nowon['geo_cent'] = Nowon_list2

# 쉬운 분석을 위한 임의의 grid id 부여
Nowon['grid_id']=0
idx = []
for i in range(len(Nowon)):
    idx.append(str(i).zfill(5))
Nowon['grid_id']=pd.DataFrame(idx)

Nowon.head()

,gid,총인구수,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,geometry,coord_cent,geo_cent,grid_id
0,다사621606,114,1.0,88,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ...","[127.0709570835209, 37.644551513114834]",POINT (127.0709570835209 37.64455151311483),00000
1,다사621606,114,1.0,60,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ...","[127.0709570835209, 37.644551513114834]",POINT (127.0709570835209 37.64455151311483),00001
2,다사619593,300,1.0,83,0.285714,73.970,0.322581,"MULTIPOLYGON (((127.06819 37.63237, 127.06819 ...","[127.06875780369337, 37.63282608884602]",POINT (127.0687578036934 37.63282608884602),00002
3,다사621619,186,1.0,69,0.100000,71.441,0.200000,"MULTIPOLYGON (((127.07033 37.65582, 127.07032 ...","[127.07088966946654, 37.65626865110605]",POINT (127.0708896694665 37.65626865110605),00003
4,다사609604,0,1.0,54,1.000000,0.000,0.000000,"MULTIPOLYGON (((127.05680 37.64225, 127.05680 ...","[127.05736572925443, 37.64269862635905]",POINT (127.0573657292544 37.64269862635905),00004


In [9]:
Nowon['정규화_디지털배움터수']=Nowon['디지털배움터수']/Nowon['디지털배움터수'].max()
Nowon['정규화_이용률']=Nowon['이용률']/Nowon['이용률'].max()
Nowon['정규화_총교육수']=Nowon['총교육수']/Nowon['총교육수'].max()
Nowon['정규화_정보화점수']=Nowon['정보화점수']/Nowon['정보화점수'].max()

Nowon.head()

,gid,총인구수,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,geometry,coord_cent,geo_cent,grid_id,정규화_디지털배움터수,정규화_이용률,정규화_총교육수,정규화_정보화점수
0,다사621606,114,1.0,88,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ...","[127.0709570835209, 37.644551513114834]",POINT (127.0709570835209 37.64455151311483),00000,1.0,1.000000,1.000000,0.842019
1,다사621606,114,1.0,60,1.000000,86.728,0.122581,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ...","[127.0709570835209, 37.644551513114834]",POINT (127.0709570835209 37.64455151311483),00001,1.0,1.000000,0.681818,0.842019
2,다사619593,300,1.0,83,0.285714,73.970,0.322581,"MULTIPOLYGON (((127.06819 37.63237, 127.06819 ...","[127.06875780369337, 37.63282608884602]",POINT (127.0687578036934 37.63282608884602),00002,1.0,0.285714,0.943182,0.718155
3,다사621619,186,1.0,69,0.100000,71.441,0.200000,"MULTIPOLYGON (((127.07033 37.65582, 127.07032 ...","[127.07088966946654, 37.65626865110605]",POINT (127.0708896694665 37.65626865110605),00003,1.0,0.100000,0.784091,0.693602
4,다사609604,0,1.0,54,1.000000,0.000,0.000000,"MULTIPOLYGON (((127.05680 37.64225, 127.05680 ...","[127.05736572925443, 37.64269862635905]",POINT (127.0573657292544 37.64269862635905),00004,1.0,1.000000,0.613636,0.000000


In [10]:
Nowon['w']=0 
Nowon['w']=(0.2686*Nowon['정규화인구']+
             0.3115*Nowon['정규화_디지털배움터수']+
             -0.2612*Nowon['정규화_이용률']+
             -0.1526*Nowon['정규화_총교육수']+
             21.3516*Nowon['정규화_정보화점수'])

In [11]:
Nowon[['gid','디지털배움터수','총교육수','이용률','정보화점수','정규화인구','geometry','grid_id','정규화_디지털배움터수','정규화_이용률','정규화_총교육수','정규화_정보화점수','w']].to_file("C:/Users/user/Desktop/노원구/Nowon.geojson",driver="GeoJSON",encoding='utf-8')

In [12]:
def generate_candidate_sites(points,M=100):
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        points: a Numpy array with shape of (N,2)
        M: the number of candidate sites to generate
    Return:
        sites: a Numpy array with shape of (M,2)
    '''
    hull = ConvexHull(points)
    polygon_points = points[hull.vertices]
    poly = Polygon(polygon_points)
    min_x, min_y, max_x, max_y = poly.bounds
    sites = []
    while len(sites) < M:
        random_point = Point([random.uniform(min_x, max_x),
                             random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])




def generate_candidate_sites(df_result_fin,M=100):
    from shapely.geometry import Polygon, Point
    sites = []
    idx=np.random.choice(np.array(range(0,len(df_result_fin))), M)
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.iloc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.iloc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])

def generate_candidate_sites(df_result_fin,Weight,M=100):
    sites = []
    idx = df_result_fin.sort_values(by = Weight, ascending = False).iloc[1:M].index
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.loc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.loc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])



from scipy.spatial import distance_matrix
def mclp(points,K,radius,M,df_result_fin,w,Weight):

    """
    Solve maximum covering location problem
    Input:
        points: input points, Numpy array in shape of [N,2]
        K: the number of sites to select
        radius: the radius of circle
        M: the number of candidate sites, which will randomly generated inside
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    print('----- Configurations -----')
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)
    print('  M %g' % M)
    import time
    start = time.time()
    sites = generate_candidate_sites(df_result_fin,Weight,M)
    J = sites.shape[0]
    I = points.shape[0]
    D = distance_matrix(points,sites)
    mask1 = D<=radius
    D[mask1]=1
    D[~mask1]=0

    from mip import Model, xsum, maximize, BINARY

    # Build model
    m = Model("mclp")
    # Add variables

    x = [m.add_var(name = "x%d" % j, var_type = BINARY) for j in range(J)]
    y = [m.add_var(name = "y%d" % i, var_type = BINARY) for i in range(I)]


    m.objective = maximize(xsum(w[i]*y[i] for i in range (I)))

    m += xsum(x[j] for j in range(J)) == K

    for i in range(I):
        m += xsum(x[j] for j in np.where(D[i]==1)[0]) >= y[i]

    m.optimize()
    
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objective_value)

    solution = []
    for i in range(J):
        if x[i].x ==1:
            solution.append(int(x[i].name[1:]))
    opt_sites = sites[solution]
            
    return opt_sites,m.objective_value

In [13]:
Nowon_fin=gpd.read_file('C:/Users/user/Desktop/노원구/Nowon.geojson')
Nowon_fin.head()

,gid,디지털배움터수,총교육수,이용률,정보화점수,정규화인구,grid_id,정규화_디지털배움터수,정규화_이용률,정규화_총교육수,정규화_정보화점수,w,geometry
0,다사621606,1.0,88,1.000000,86.728,0.122581,00000,1.0,1.000000,1.000000,0.842019,17.909087,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ..."
1,다사621606,1.0,60,1.000000,86.728,0.122581,00001,1.0,1.000000,0.681818,0.842019,17.957642,"MULTIPOLYGON (((127.07039 37.64410, 127.07039 ..."
2,다사619593,1.0,83,0.285714,73.970,0.322581,00002,1.0,0.285714,0.943182,0.718155,15.513353,"MULTIPOLYGON (((127.06819 37.63237, 127.06819 ..."
3,다사621619,1.0,69,0.100000,71.441,0.200000,00003,1.0,0.100000,0.784091,0.693602,15.028959,"MULTIPOLYGON (((127.07033 37.65582, 127.07032 ..."
4,다사609604,1.0,54,1.000000,0.000,0.000000,00004,1.0,1.000000,0.613636,0.000000,-0.043341,"MULTIPOLYGON (((127.05680 37.64225, 127.05680 ..."


In [14]:
# 100X100 grid에서 central point 찾기
Nowon_list = []
Nowon_list2 = []
for i in Nowon_fin['geometry']:
    cent = [[i[0].centroid.coords[0][0],i[0].centroid.coords[0][1]]]
    Nowon_list.append(cent)
    Nowon_list2.append(Point(cent[0]))
Nowon_fin['coord_cent'] = 0
Nowon_fin['geo_cent'] = 0
Nowon_fin['coord_cent']= pd.DataFrame(Nowon_list) # pydeck을 위한 coordinate type
Nowon_fin['geo_cent'] = Nowon_list2 # geopandas를 위한 geometry type

In [15]:
points = []
for i in Nowon_fin['coord_cent'] :
    points.append(i)

w= []
for i in Nowon_fin['w'] :
    w.append(i)

radius = radius = (1/88.74/1000)*300  #반경?   
K = 5     # 입지 선정 개수
M = 5000  #후보 입지 수

opt_sites_org,f = mclp(np.array(points),K,radius,M,Nowon_fin,w,'w')


Nowon_opt= pd.DataFrame(opt_sites_org)
Nowon_opt.columns = ['lon', 'lat']
Nowon_opt

----- Configurations -----
  Number of points 3744
  K 5
  Radius 0.00338066
  M 5000
----- Output -----
  Running time : 85.81147694587708 seconds
  Optimal coverage points: 2439.7


,lon,lat
0,127.073121,37.662586
1,127.067483,37.657158
2,127.078998,37.625652
3,127.074491,37.621129
4,127.054889,37.677841


In [16]:
Nowon_opt.to_csv('C:/Users/user/Desktop/노원구/노원후보지_5개.csv',index=False)